In [1]:
# Energy and Buildings
idNo = '271089'

In [2]:
import requests
from bs4 import BeautifulSoup
from pandas.io.html import read_html
import pandas as pd
import json
import csv

In [3]:
df = pd.DataFrame()
df.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

df = pd.read_csv("blankFile.csv", names= columnNames , header = None)
df = df.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    df[section] = ''
    df[sectionData] = ''

df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19


In [4]:
df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19


In [5]:
# searchTerm1 = ['Building Information Modelling']
# searchTerm2 = ['Green Buildings']

# searchTerm1 = ['BIM']
# searchTerm2 = ['Sustainability']

tc = 0

searchTerm1 = ['BIM' , 'Building Information Modelling','Building information model','6D BIM', 'Digital construction','Smart construction','Digital engineering']
searchTerm2 = ['Green Building' , 'Green Buildings','Sustainable building','Sustainable buildings','Environmentally friendly design' , 'Sustainability' , 'Sustainable development' , 'Energy','Water' ,'Waste' , 'Thermal comfort','Electricity' ,'Solar','Wind power' , 'Green infrastructure','Life cycle assessment']

counter = 0
programSearchTerms1 = []
programSearchTerms2 = []

for term2 in searchTerm2:
    text2 = ""
    terms2 = term2.split(' ')
    for t2 in terms2:
        text2 = text2 + t2 + "%20"

    fileIndex = text2.rfind("%20")
    te = text2[:fileIndex]
    programSearchTerms2.append(te)

print(programSearchTerms2)

for term1 in searchTerm1:
    text1 = ""
    terms1 = term1.split(' ')
    for t2 in terms1:
        text1 = text1 + t2 + "%20"

    fileIndex = text1.rfind("%20")
    te = text1[:fileIndex]
    programSearchTerms1.append(te)

print(programSearchTerms1)

['Green%20Building', 'Green%20Buildings', 'Sustainable%20building', 'Sustainable%20buildings', 'Environmentally%20friendly%20design', 'Sustainability', 'Sustainable%20development', 'Energy', 'Water', 'Waste', 'Thermal%20comfort', 'Electricity', 'Solar', 'Wind%20power', 'Green%20infrastructure', 'Life%20cycle%20assessment']
['BIM', 'Building%20Information%20Modelling', 'Building%20information%20model', '6D%20BIM', 'Digital%20construction', 'Smart%20construction', 'Digital%20engineering']


In [6]:
def getData(link):
    browser = webdriver.Safari()
    browser.get(link)
    respData = browser.page_source
    browser.quit()
    parsed_html = BeautifulSoup(respData)
    
    title = parsed_html.find('span',{'class':'title-text'}).text
    print("Title: ",title)
    df.loc[row,'url'] = link
    df.loc[row,'Title'] = title
    
    absResults = parsed_html.find('div',{'class':'abstract author'})
    
    if absResults is not None:
        try:            
            df.loc[row,'section0'] = absResults.find('h2').text
            df.loc[row,'sectionData0'] = absResults.find('div').text
        except:
            df.loc[row,'section0'] = "None"
            df.loc[row,'sectionData0'] = "None"
    else:
        df.loc[row,'section0'] = "None"
        df.loc[row,'sectionData0'] = "None"

        
    sections = parsed_html.find('div',{'class':'Body u-font-serif'})
    if sections is not None:        
        sections = sections.find_next('div')
        count = 1 
        for section in sections:
            if section is not None: 
                sectionNo = 'section' + str(count)
                sectionData = 'sectionData' + str(count)
                try:
                    df.loc[row,sectionNo] = section.find('h2',{'class':'u-h3 u-margin-l-top u-margin-xs-bottom'}).text
                except:
                    df.loc[row,sectionNo] = "Some issue"
                    
                data = ''
                for para in section.findAll('p'):
                    data = data + para.text
                df.loc[row,sectionData] = data
            else:
                df.loc[row,sectionNo] = "None"
            count = count + 1        

In [7]:
from bs4 import BeautifulSoup
from selenium import webdriver

In [8]:
baseUrl = 'https://www.sciencedirect.com/search/api?tak=%22'

querySet = []
row = 0
for item1 in programSearchTerms1:
    for item2 in programSearchTerms2:
        sq = 'https://www.sciencedirect.com/search?qs=%22' + item1 + '%22%20%2B%20%22' + item2 + '%22'
        print("base query: ",sq)
        query = baseUrl + item1 +"%22%20%2B%20%22"+ item2+"%22&show=25&publicationTitles=" + idNo + "&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com"
        
        # Call webdriver         
        browser = webdriver.Safari()
        browser.get(query)
        respData = browser.page_source
        browser.quit()

        # Clear html         
        respData = respData.replace('<html><head></head><body><pre style="word-wrap: break-word; white-space: pre-wrap;">',"")
        respData = respData.replace('</pre></body></html>',"")
        
        py_dic = json.loads(respData)
        
        # Finding the number of pages in the selected journals.         
        for key in py_dic:
            if key == 'resultsFound':
                pageResults = py_dic[key]
        print("pageResults: ",pageResults)                
                                
        if pageResults > 25:
            page = pageResults // 25
            lastPage = pageResults % 25
            if lastPage > 0:
                page = page + 1
        else:
            page = 1
        print("page: ",page)

        #         for p in range(2):        
        for p in range(page):
            print("In page: ",p)
            if p == 0:
                query = baseUrl + item1 +"%22%20%2B%20%22"+ item2+"%22&show=25&publicationTitles=" + idNo + "&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com"
            else:
                offset = '&offset='+str(p*25)
                query = baseUrl + item1 +"%22%20%2B%20%22"+ item2+"%22&show=25&publicationTitles=" + idNo + "&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com"
                
            print("search query: ",query)  
                
            # Call webdriver         
            browser = webdriver.Safari()
            browser.get(query)
            respData = browser.page_source
            browser.quit()
            # Load Json Data 

            respData = respData.replace('<html><head></head><body><pre style="word-wrap: break-word; white-space: pre-wrap;">',"")
            respData = respData.replace('</pre></body></html>',"")

            py_dic = json.loads(respData)                
            # Quit the browser 
            links = []
            for key in py_dic:
                if key == 'searchResults':
                    for searchR in py_dic[key]:
                        for search in searchR:
                            if search == 'pdf':
                                for linkR in searchR[search]:
                                    if linkR == 'getAccessLink':
                                        baseLink = 'https://www.sciencedirect.com/'
                                        link = baseLink +searchR[search][linkR]
                                        print('Reading result: ',row)
                                        df.loc[row,'searchTerm1'] = item1
                                        df.loc[row,'searchTerm2'] = item2
                                        getData(link)                                    
                                        row = row + 1        

base query:  https://www.sciencedirect.com/search?qs=%22BIM%22%20%2B%20%22Green%20Building%22
pageResults:  3
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22BIM%22%20%2B%20%22Green%20Building%22&show=25&publicationTitles=271089&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
Reading result:  0
Title:  Continuous-time Bayesian calibration of energy models using BIM and energy data
Reading result:  1
Title:  A new method for reusing building information models of past projects to optimize the default configuration for performance simulations
Reading result:  2
Title:  Evaluating the impact of operating energy reduction measures on embodied energy
base query:  https://www.sciencedirect.com/search?qs=%22BIM%22%20%2B%20%22Green%20Buildings%22
pageResults:  3
page:  1
In page:  0
search query

Title:  Parametric modeling and surface-specific sensitivity analysis of PV module layout on building skin using BIM
Reading result:  37
Title:  Evaluating the impact of operating energy reduction measures on embodied energy
Reading result:  38
Title:  Critical review of bim-based LCA method to buildings
Reading result:  39
Title:  An integrated BIM-based framework for minimizing embodied energy during building design
Reading result:  40
Title:  BIM application to building energy performance visualisation and management: Challenges and potential
Reading result:  41
Title:  Framework for a UAS-based assessment of energy performance of buildings
Reading result:  42
Title:  Computational geometry in the context of building information modeling
Reading result:  43
Title:  BPOpt: A framework for BIM-based performance optimization
Reading result:  44
Title:  Cost-effective energy saving measures based on BIM technology: Case study at National Taiwan University
Reading result:  45
Title:  A v

Title:  Transformation of a university building into a zero energy building in Mediterranean climate
base query:  https://www.sciencedirect.com/search?qs=%22BIM%22%20%2B%20%22Wind%20power%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22BIM%22%20%2B%20%22Wind%20power%22&show=25&publicationTitles=271089&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%22BIM%22%20%2B%20%22Green%20infrastructure%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22BIM%22%20%2B%20%22Green%20infrastructure%22&show=25&publicationTitles=271089&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6Wv

base query:  https://www.sciencedirect.com/search?qs=%22Building%20Information%20Modelling%22%20%2B%20%22Energy%22
pageResults:  28
page:  2
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Building%20Information%20Modelling%22%20%2B%20%22Energy%22&show=25&publicationTitles=271089&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
Reading result:  112
Title:  Life-cycle building information modelling (BIM) engaged framework for improving building energy performance
Reading result:  113
Title:  Computational geometry in the context of building information modeling
Reading result:  114
Title:  Continuous-time Bayesian calibration of energy models using BIM and energy data
Reading result:  115
Title:  The promise of BIM for improving building performance
Reading result:  116
Title:  A BIM based tool for

Title:  Transformation of a university building into a zero energy building in Mediterranean climate
base query:  https://www.sciencedirect.com/search?qs=%22Building%20Information%20Modelling%22%20%2B%20%22Electricity%22
pageResults:  3
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Building%20Information%20Modelling%22%20%2B%20%22Electricity%22&show=25&publicationTitles=271089&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
Reading result:  164
Title:  A verification study for energy analysis of BIPV buildings with BIM
Reading result:  165
Title:  An integrated BIM-based framework for the optimization of the trade-off between embodied and operational energy
Reading result:  166
Title:  BIM-based techno-economic assessment of energy retrofitting residential buildings in hot humid climat

...

Title:  A method of probabilistic risk assessment for energy performance and cost using building energy simulation
base query:  https://www.sciencedirect.com/search?qs=%22Building%20information%20model%22%20%2B%20%22Water%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Building%20information%20model%22%20%2B%20%22Water%22&show=25&publicationTitles=271089&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%22Building%20information%20model%22%20%2B%20%22Waste%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Building%20information%20model%22%20%2B%20%22Waste%22&show=25&publicationTitles=271089&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQL

base query:  https://www.sciencedirect.com/search?qs=%226D%20BIM%22%20%2B%20%22Sustainability%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%226D%20BIM%22%20%2B%20%22Sustainability%22&show=25&publicationTitles=271089&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%226D%20BIM%22%20%2B%20%22Sustainable%20development%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%226D%20BIM%22%20%2B%20%22Sustainable%20development%22&show=25&publicationTitles=271089&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base que

base query:  https://www.sciencedirect.com/search?qs=%22Digital%20construction%22%20%2B%20%22Sustainable%20development%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Digital%20construction%22%20%2B%20%22Sustainable%20development%22&show=25&publicationTitles=271089&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%22Digital%20construction%22%20%2B%20%22Energy%22
pageResults:  1
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Digital%20construction%22%20%2B%20%22Energy%22&show=25&publicationTitles=271089&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D

base query:  https://www.sciencedirect.com/search?qs=%22Smart%20construction%22%20%2B%20%22Sustainable%20development%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Smart%20construction%22%20%2B%20%22Sustainable%20development%22&show=25&publicationTitles=271089&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%22Smart%20construction%22%20%2B%20%22Energy%22
pageResults:  1
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Smart%20construction%22%20%2B%20%22Energy%22&show=25&publicationTitles=271089&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostnam

base query:  https://www.sciencedirect.com/search?qs=%22Digital%20engineering%22%20%2B%20%22Sustainable%20development%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Digital%20engineering%22%20%2B%20%22Sustainable%20development%22&show=25&publicationTitles=271089&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%22Digital%20engineering%22%20%2B%20%22Energy%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Digital%20engineering%22%20%2B%20%22Energy%22&show=25&publicationTitles=271089&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hos

In [9]:
df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19
0,BIM,Green%20Building,https://www.sciencedirect.com//science/article...,Continuous-time Bayesian calibration of energy...,Abstract,The present study proposes a framework for the...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BIM,Green%20Building,https://www.sciencedirect.com//science/article...,A new method for reusing building information ...,Abstract,The purpose of this study is to develop a new ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BIM,Green%20Building,https://www.sciencedirect.com//science/article...,Evaluating the impact of operating energy redu...,Abstract,"Annually, 48% of the global energy is used by ...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BIM,Green%20Buildings,https://www.sciencedirect.com//science/article...,Continuous-time Bayesian calibration of energy...,Abstract,The present study proposes a framework for the...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BIM,Green%20Buildings,https://www.sciencedirect.com//science/article...,A new method for reusing building information ...,Abstract,The purpose of this study is to develop a new ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,Building%20information%20model,Life%20cycle%20assessment,https://www.sciencedirect.com//science/article...,An integrated BIM-based framework for minimizi...,Abstract,Assessment of the embodied energy associated w...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257,Building%20information%20model,Life%20cycle%20assessment,https://www.sciencedirect.com//science/article...,Embodied and operational energy for new-build ...,Abstract,In this study a Building Information Model (BI...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
258,Digital%20construction,Energy,https://www.sciencedirect.com//science/article...,Energy-saving potential of 3D printed concrete...,Abstract,Large-scale concrete 3D printing and digital c...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259,Digital%20construction,Thermal%20comfort,https://www.sciencedirect.com//science/article...,Energy-saving potential of 3D printed concrete...,Abstract,Large-scale concrete 3D printing and digital c...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df1 = df
df['journalName'] = str(idNo)
fileSave = 'kgRawData/SD/sd-kg'+idNo+'.csv'
df.to_csv(fileSave)

NameError: name 'df' is not defined

In [27]:
import pandas as pd
data = pd.read_csv('kgRawData/SD/sd-kg271089.csv')
data['presence'] = False

In [28]:
searchTerm1 = ['BIM' , 'Building Information Modelling','Building information model','6D BIM', 'Digital construction','Smart construction','Digital engineering']
searchTerm2 = ['Green Building' , 'Green Buildings','Sustainable building','Sustainable buildings','Environmentally friendly design' , 'Sustainability' , 'Sustainable development' , 'Energy','Water' ,'Waste' , 'Thermal comfort','Electricity' ,'Solar','Wind power' , 'Green infrastructure','Life cycle assessment']

validEntries = []

for i in range(data.shape[0]):    
    for st1 in searchTerm1:
        for st2 in searchTerm2:
            if (st1 in data['sectionData0'][i]) and (st2 in data['sectionData0'][i]):
                data['presence'][i] = True
                validEntries.append(i)

<ipython-input-28-b50795caeeff>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['presence'][i] = True


In [35]:
uniqueValidEntries = list(set(validEntries))

In [36]:
uniqueDf = pd.DataFrame()
uniqueDf.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

uniqueDf = pd.read_csv("blankFile.csv", names= columnNames , header = None)
uniqueDf = uniqueDf.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    uniqueDf[section] = ''
    uniqueDf[sectionData] = ''

In [44]:
for entry in range(len(uniqueValidEntries)):
    uniqueDf.loc[entry] = data.loc[uniqueValidEntries[entry]]

In [46]:
uniqueDf['url'][0]

'https://www.sciencedirect.com//science/article/pii/S0378778820300116'

In [47]:
uniqueDf

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19
0,Building%20Information%20Modelling,Energy,https://www.sciencedirect.com//science/article...,BIM-based techno-economic assessment of energy...,Abstract,Buildings in hot humid climates are energy int...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Building%20Information%20Modelling,Energy,https://www.sciencedirect.com//science/article...,BIM interface for full vs. semi-automated buil...,Abstract,BIM (building information model) enables infor...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Building%20Information%20Modelling,Energy,https://www.sciencedirect.com//science/article...,Quantitative approach for evaluating the build...,Abstract,Architectural building design practic in hot c...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BIM,Sustainability,https://www.sciencedirect.com//science/article...,BIM-based techno-economic assessment of energy...,Abstract,Buildings in hot humid climates are energy int...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Building%20Information%20Modelling,Energy,https://www.sciencedirect.com//science/article...,An integrated BIM-based framework for the opti...,Abstract,Design choices with a unilateral focus on the ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,BIM,Sustainability,https://www.sciencedirect.com//science/article...,Quantitative approach for evaluating the build...,Abstract,Architectural building design practic in hot c...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,BIM,Sustainability,https://www.sciencedirect.com//science/article...,An integrated BIM-based framework for the opti...,Abstract,Design choices with a unilateral focus on the ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Building%20Information%20Modelling,Energy,https://www.sciencedirect.com//science/article...,BIM-based techno-economic assessment of energy...,Abstract,Buildings in hot humid climates are energy int...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Building%20Information%20Modelling,Energy,https://www.sciencedirect.com//science/article...,BIM interface for full vs. semi-automated buil...,Abstract,BIM (building information model) enables infor...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Building%20Information%20Modelling,Energy,https://www.sciencedirect.com//science/article...,Quantitative approach for evaluating the build...,Abstract,Architectural building design practic in hot c...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
uniqueDf.to_csv('kgRawData/SD/cleanSd-kg271089.csv')

In [50]:
len(uniqueDf)

39